In [1]:
import os
import tempfile
import urllib
import pprint

import absl
import tensorflow as tf
import tfx

tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import types
#from tfx.utils.dsl_utils import csv_input
from tfx.components import CsvExampleGen
#from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from  tfx.proto import example_gen_pb2
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ModelValidator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import external_input

In [2]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.0.0
TFX version: 0.15.0


Use cell magic %%skip_for_export to denote cells to skip during export to pipeline.

In [3]:
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

## Set up pipeline paths and logging

In [4]:
# Set up paths.
#_tfx_root = tfx.__path__[0]
_tfx_root = os.getcwd()

_stock_root = os.path.join(_tfx_root, 'data/stock_pipeline')
# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
#_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model/stock_simple')
_serving_model_dir = os.path.join(_tfx_root, 'serving_model/stock_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

_data_root = os.path.join(_tfx_root, 'data/input/small')


## Create the InteractiveContext

In [5]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()


## Run TFX components interactively

Next, we construct TFX components and run each one interactively using within the interactive session to obtain ExecutionResult objects.


### ExampleGen

ExampleGen brings data into the TFX pipeline.



In [6]:
%%time
# Use the packaged CSV input data.
examples = external_input(_data_root)

# Input train split is '_data_root/train/*', eval split is '_data_root/eval/*'
# test split is '_data_root/test/*'.
# Output splits are generated one-to-one mapping from input splits.
input = example_gen_pb2.Input(splits=[
                example_gen_pb2.Input.Split(name='train', pattern='train/*'),
                example_gen_pb2.Input.Split(name='eval', pattern='eval/*')
                #example_gen_pb2.Input.Split(name='test', pattern='test/*')
            ])

# Brings data into the pipeline or otherwise joins/converts training data.
example_gen = CsvExampleGen(input=examples, input_config=input)
context.run(example_gen)


INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 3.
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data /home/denis/ML/tfx/stocks/data/input/small/train/* to TFExample.
INFO:absl:Processing input csv data /home/denis/ML/tfx/stocks/data/input/small/eval/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


CPU times: user 23.5 s, sys: 95.9 ms, total: 23.6 s
Wall time: 24.6 s


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/CsvExampleGen/examples/1/train/, split: train, id: 2)
            Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/CsvExampleGen/examples/1/eval/, split: eval, id: 3)]
        ))

In [7]:
%%skip_for_export

for artifact in example_gen.outputs['examples'].get():
  print(artifact.split, artifact.uri)

train /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/CsvExampleGen/examples/1/train/
eval /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/CsvExampleGen/examples/1/eval/
This cell will be skipped during export to pipeline.


In [8]:
%%skip_for_export

import tensorflow_data_validation as tfdv

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = example_gen.outputs['examples'].get()[0].uri

def show_data(_uri, _count):
    # Get the list of files in this directory (all compressed TFRecord files)
    tfrecord_filenames = [os.path.join(_uri, name)
                          for name in os.listdir(_uri)]

    # Create a `TFRecordDataset` to read these files
    dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
    decoder = tfdv.TFExampleDecoder()

    # Iterate over the first 3 records and decode them using a TFExampleDecoder
    for tfrecord in dataset.take(_count):
      serialized_example = tfrecord.numpy()
      example = decoder.decode(serialized_example)
      pp.pprint(example)
    
show_data(train_uri, 3)

{'Nr': array([0]),
 'close': array([114.56], dtype=float32),
 'cross_idx': array([0.], dtype=float32),
 'date': array([b'2018-01-02'], dtype=object),
 'dir': array([0.], dtype=float32),
 'high': array([114.6025], dtype=float32),
 'low': array([114.54], dtype=float32),
 'open': array([114.6025], dtype=float32),
 'sum_vol': array([1119]),
 'time': array([b'09:30:38'], dtype=object)}
{'Nr': array([6]),
 'close': array([114.61], dtype=float32),
 'cross_idx': array([0.], dtype=float32),
 'date': array([b'2018-01-02'], dtype=object),
 'dir': array([0.], dtype=float32),
 'high': array([114.61], dtype=float32),
 'low': array([114.558], dtype=float32),
 'open': array([114.56], dtype=float32),
 'sum_vol': array([1017]),
 'time': array([b'09:31:54'], dtype=object)}
{'Nr': array([11]),
 'close': array([114.62], dtype=float32),
 'cross_idx': array([0.], dtype=float32),
 'date': array([b'2018-01-02'], dtype=object),
 'dir': array([0.], dtype=float32),
 'high': array([114.62], dtype=float32),
 'low':

### StatisticsGen (using Tensorflow Data Validation)

In [9]:
%%time
# Computes statistics over data for visualization and example validation.
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 5.
INFO:absl:Running executor for StatisticsGen
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Generating statistics for split train
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/StatisticsGen/output/2/train/.
INFO:absl:Generating statistics for split eval
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/StatisticsGen/output/2/eval/.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


CPU times: user 8.28 s, sys: 177 ms, total: 8.46 s
Wall time: 8.62 s


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        output: Channel(
            type_name: ExampleStatisticsPath
            artifacts: [Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/StatisticsGen/output/2/train/, split: train, id: 4)
            Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/StatisticsGen/output/2/eval/, split: eval, id: 5)]
        ))

In [10]:
%%skip_for_export

context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


This cell will be skipped during export to pipeline.



## SchemaGen (using Tensorflow Data Validation)

SchemaGen generates a schema for your data based on computed statistics. This component also uses the Tensorflow Data Validation library.


In [11]:
# Generates schema based on statistics files.
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 7.
INFO:absl:Running executor for SchemaGen
INFO:absl:Infering schema from statistics.
INFO:absl:Schema written to /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/SchemaGen/output/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        output: Channel(
            type_name: SchemaPath
            artifacts: [Artifact(type_name: SchemaPath, uri: /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/SchemaGen/output/3/, split: , id: 6)]
        ))

In [12]:
schema_gen.outputs['schema']

Channel(
    type_name: SchemaPath
    artifacts: [Artifact(type_name: SchemaPath, uri: /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/SchemaGen/output/3/, split: , id: 6)]
)

In [13]:
%%skip_for_export
#Visualize the inferred schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'date',BYTES,required,single,-
'time',BYTES,required,single,-
'Nr',INT,required,single,-
'close',FLOAT,required,single,-
'cross_idx',FLOAT,required,single,-
'dir',FLOAT,required,single,-
'high',FLOAT,required,single,-
'low',FLOAT,required,single,-
'open',FLOAT,required,single,-


This cell will be skipped during export to pipeline.



## ExampleValidator

ExampleValidator performs anomaly detection based on computed statistics and your data schema.


In [14]:
# Performs anomaly detection based on statistics and data schema.
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'], 
                                     schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 9.
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics.
INFO:absl:Validation complete. Anomalies written to /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/ExampleValidator/output/4/.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        output: Channel(
            type_name: ExampleValidationPath
            artifacts: [Artifact(type_name: ExampleValidationPath, uri: /tmp/tfx-interactive-2020-01-19T14_06_22.205831-7b1efojm/ExampleValidator/output/4/, split: , id: 7)]
        ))

In [15]:
%%skip_for_export

context.show(example_validator.outputs['anomalies'])

This cell will be skipped during export to pipeline.


## Transform

Transform performs data transformations and feature engineering which are kept in sync for training and serving.


In [ ]:
_stock_constants_module_file = 'stock_constants.py'

In [ ]:
%%skip_for_export
%%writefile {_stock_constants_module_file}

DENSE_FLOAT_FEATURE_KEYS = ['close', 'open', 'high', 'low', 'sum_vol']

# Keys
LABEL_KEY = 'dir'

def transformed_name(key):
  return key + '_xf'

In [ ]:
_stock_transform_module_file = 'stock_transform.py'

In [ ]:
%%skip_for_export
%%writefile {_stock_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import stock_constants

_DENSE_FLOAT_FEATURE_KEYS = stock_constants.DENSE_FLOAT_FEATURE_KEYS
_LABEL_KEY = stock_constants.LABEL_KEY
_transformed_name = stock_constants.transformed_name

def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
    
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.scale_to_0_1(
        _fill_in_missing(inputs[key]))

  outputs[_transformed_name(_LABEL_KEY)] = _fill_in_missing(inputs[_LABEL_KEY])

  return outputs

def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

#### Run the Transform component


In [ ]:
# Performs transformations and feature engineering in training and serving.
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_stock_transform_module_file))
context.run(transform)

In [ ]:
context.show(transform.outputs)

In [ ]:
train_uri = transform.outputs['transform_output'].get()[0].uri
os.listdir(train_uri)

In [ ]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = transform.outputs['transformed_examples'].get()[1].uri
show_data(train_uri, 3)


## Trainer
The Trainer component will train a model that you define in TensorFlow (either using the Estimator API or the Keras API with model_to_estimator).